In [ ]:
from keras.datasets import mnist
(X_train,y_train),(X_test,y_test) = mnist.load_data()

In [ ]:
#Playing with shape
#x_temp=X_train
#x_temp.reshape(-1,60000).shape
x_temp = None

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import numpy as np

In [ ]:
for i in range(0,6):
    plt.subplot(2,3,i+1)
    plt.imshow(X_train[i],cmap="gray")
    plt.title(str(y_train[i]))
    plt.axis("off")
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(20,20))
for i in range(6):
    ax = fig.add_subplot(1,6,i+1,xticks=[],yticks=[])
    ax.imshow(X_train[i],cmap="gray")
    ax.set_title(str(y_train[i]))

In [ ]:
#rescale [0,255] -> [0,1]
X_train = X_train.astype("float32")/255
X_test = X_test.astype("float32")/255

In [ ]:
#OnehotLabel
import pandas as pd
y_train=np.array(pd.get_dummies(y_train))

In [ ]:
##Reshaping X_train
X_train_matrix = X_train.reshape(60000,-1)
X_train_matrix.shape

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense,Activation
model = Sequential()
model.add(Dense(32,input_dim=X_train_matrix.shape[1]))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary()

In [ ]:
model.fit(X_train_matrix,y_train,epochs=200,batch_size=124,verbose=1)
score = model.evaluate(X_train_matrix,y_train)

In [ ]:
X_test_flatten = X_test.reshape(X_test.shape[0],-1)
y_test_one_hot = np.array(pd.get_dummies(y_test))

In [ ]:
score_test = model.evaluate(X_test_flatten,y_test_one_hot)
score_test[1]

In [ ]:
#Model in Lecture
from keras.layers import Dropout
model1 = Sequential()
model1.add(Dense(512,input_dim=X_train_matrix.shape[1]))
model1.add(Dropout(0.5))
model1.add(Dense(512))
model1.add(Dropout(0.7))
model1.add(Dense(10,activation="softmax"))
model1.summary()

In [ ]:
model1.compile(loss="categorical_crossentropy",optimizer="rmsprop",metrics=["accuracy"])
score_1 = model1.evaluate(X_test_flatten,y_test_one_hot)
100*score_1[1]

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath="./best_weight_mnist.hdf5",verbose=1,
                              save_best_only=True)
hist = model1.fit(X_train_matrix,y_train,batch_size=128,epochs=6,validation_split=0.2,
                 callbacks=[checkpointer],verbose=1,shuffle=True)

In [ ]:
score_1 = model1.evaluate(X_test_flatten,y_test_one_hot)
100*score_1[1]

# Convolution Neural Network

In [ ]:
##OK WITH Lossing DATA then PADDING = "VALID"
# IF NOT OK WITH LOSSING THE DATA THEN PADDING = "SAME"
#Depends on the stride of our filter



In [ ]:
from keras.layers import Conv2D
Conv2D(filters,kernel_size,strides,padding,activation="relu",input_shape)
#filters - the number of filter
#Kernel_size - number specifying both height and width of (square)
#convolution window.
#activation-typically relu.if not specify anything then no activation function
#strongly encouraged to specify relu in activation 
#strides-the stride of convolution , if not specified then default is 1.
#padding - one of "valid" or "same" if nothing is not specified then by default is set to "valid"
#NOTE : it is possible to reperesent both kernel_size and strides as either a number or a tuple
#When using your convolution layer as the first layer (appearing after the input layer)in a model,we must provide a input shape
#And donot specify a input shape if its not the first layer.

# Example #1

In [ ]:
#Say I'm constructing a CNN and my input layer accepts a grayscale images that are 
#200by200 pixels(corresponding to a 3D array with height 200, width 200 depth 1).Then
#say I'd like the next layer to be a convolution layer with 16 filters,each with width and 
#height of 3. When performing the convoultion, I'd like the filter to jump two pixels
#at a time. I also don't want the filter to extend outside of the image bondaries 
#in other words,I don't want to pad the images with zeros. Then, to construct this convolution 
#layer, I would use the following line of command



Conv2D(filters=16,kernel_size=2,strides=2,activation="relu",input_shape=(200,200,1))


# Example #2

In [ ]:
# Say I'd like the next layer in my CNN to be a convolution layer that takes 
#the layer constructed in Example 1 as input. Say I'd like my new layer to have 32 
#filters,each with a height and width of 3.When performing convolution, I'd like the filter 
#to jump 1 pixels at a time. I want the convolution layer to see all regions of previous layer, and 
#so I don't mind if the filter hangs over the edge of previous layer when its performing
#the convolution. Then, to construct this convolution layer I would use following
#line of code
Conv2D(filters=32,kernel_size=3,padding="same",activation="relu")

# Example #3

In [ ]:
#In this case, there are 64 filters and the size of filter is (2x2), and the 
#layer has a Relu activation function.The other arguments in the layer use the
#default values, so this convolution uses a stride of 1 and the padding is set to
#"valid"(so no padding)
Conv2D(64,(2,2),activation="relu")

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
model = Sequential()
model.add(Conv2D(filters=16,kernel_size=2,strides=2,padding="valid",activation="relu",
                input_shape = (200,200,1)))
model.summary()

In [ ]:
## Take note how the number of parameter in the convolution layer changes
## Params # 80 
## also notice the shape of the convolution layer changes . This corresponds
# to value under Output shape in the printed output.
# None corresponds to the batch size, and the convolution layer has. a height of 100,
# width of 100 and depth of 16


# Formula: Number of Parameter in a convolution Layer

Number of parameter in a covolution layer depends on the supplied values of **filters**, **kernel_size** and **input_shape**.
K - the number of filter in a convolution layer
F -  the height and width of convolution filters
D_in - the depth of previous layer
Notice that K = filters and F=kernel_size . likewise D_in the last value in the input_shape tuple.
Since there are (**F x F x D_in**) weights per filter, and the convolution layer is composed of K filters, the total number of weights in convolution layer is equal to **K x F x F x D_in** and there is a bias term per filter thus the number of parameters in convolution layer is give by **(K x F x F x D_in + K)**



In [ ]:
2*2*16*1 + 16
## Above example number of parameters

# Formula: Shape of a Convolution Layer

The shape of a convoution layer depends on the supplied values of **kernel_size, input_shape, padding** and **stride**.
* K - the number of filter in the convolution layer 
* F - the height and width of convolution filters
* S - the stride of convolution filters
* H_in - the height of the previous layer
* W_in - the width of the previous layer
Notice that K = filters , F = kernel_size and S = stride and H_in and W_in is equal to first and second value of input_shape respectively.
* The depth of convolution layer will be always be equal to **K**(no of filter)
IF **padding = "same"**, then spatial dimension of the convolution layer are the following
* height = ceil(float(H_in)/float(S))
* width = ceil(float(W_in)/float(S))
If **padding = "valid"**, then spatial dimension of the convolution layer are the following:
* height = ceil(float(H_in - F +1)/float(S))
* width = ceil(float(W_in - F +1)/float(S))




# Quiz

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=3,strides=2,padding="same",
                activation = "relu",input_shape=(128,128,3)))
model.summary()
#Depth of convolution layer is equal to number of filter

# Polling of layers!

# Max Pooling layer

In [ ]:
# So if want to find a complex pattern in our data then we will have many filters 
#and dimentionalty we will increase thus it will overfit our training data
#THAT is the reason we introduce polling layer.
#first we will discuss about max polling layer 
#Max Pooling Layer
#Window Size: 2*2
#Strides 2
# So it reduces the dimension of convolution layer (Half of original size)



# Global average Pooling Layer!

In [ ]:
# A global average pooling layer takes a 3D array and turns it into a vector.


## Max Pooling Layers in Keras

In [ ]:
from keras.layers import MaxPooling2D
MaxPooling2D(pool_size=2,strides=2,padding="valid")

## Arguments
* pool_size - Number specifying the height and width of the pooling window
* strides - The vertical and horizontal stride. IF don't specify anything,strides will default to pool_size
* padding - One of the "valid" or "same". If don't specify anything padding is set to "valid"
Input could be number or tuple

## Example 
Say I'm constructing a CNN, and I'd like to reduce the dimentionality of a convolution layer by following it with a max pooling layer.Say the convolution layer has size **(100,100,15)** and I'd like the max pooling layer to have a size **(50,50,15)**. I can do this by using 2 x 2 window in my max pooling layer, with a stride of 2,which could be constructed in the following line of code.

In [ ]:
MaxPooling2D(pool_size=2,strides=2)
# IF you'd like to use a stride of 1, but still keep the size of window at 
#2 x 2 then we could use
MaxPooling2D(pool_size=2,strides=1)

## Checking the Dimentionality of Max Pooling Layers


In [ ]:
from keras.layers import MaxPooling2D
from keras.models import Sequential
model = Sequential()
model.add(MaxPooling2D(pool_size=2,strides=2,input_shape=(100,100,15)))
model.summary()

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
model = Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",
                input_shape=(32,32,3)))
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu",))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu",))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu",))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dense(10,activation="softmax"))
model.summary()

## Things to Remember
* Always add a ReLU activation to Conv2D layer in your CNN. With the exception of the final layer in the network, Dense layers should also have a ReLU activation function
* When constructing a network for classification, the final layer in the network should be a dense layer with a Softmax activation function. The number of nodes in the final layer should be equal the total number of classes in the dataset.


# CIFAR-10 --CNNs

In [ ]:
import keras 
from keras.datasets import cifar10

# LOAD THE PRE -SHUFFLED TRAIN AND TEST DATA
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline 
fig = plt.figure(figsize=(20,5))
for i in range(36):
    ax = fig.add_subplot(3,12,i+1,xticks=[],yticks=[])
    ax.imshow(np.squeeze(x_train[i])) 

In [ ]:
#Normalizing the data
x_train = x_train.astype("float32")/255
x_test=x_test.astype("float32")/255

In [ ]:
from keras.utils import np_utils
import pandas as pd
#one hot encoding in keras
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,10)

In [ ]:
#break data into training and validation set
(x_train,x_valid)=x_train[5000:],x_train[:5000]
(y_train,y_valid)=y_train[5000:],y_train[:5000]

In [ ]:
#print(x_train.shape)
#print(y_train.shape)

In [ ]:
#Infereing
print(x_train.shape)
print(y_valid.shape)

In [ ]:
#print number of training, validation ,and test images
print(x_train.shape[0],"training samples")
print(x_test.shape[0],"testing samples")
print(x_valid.shape[0],"validating samples")

# First Model-- Regular Vanilla Neural Network!

In [ ]:
#falttening of image 
x_train_vanilla = x_train.reshape(x_train.shape[0],-1)
x_valid_vanilla = x_valid.reshape(x_valid.shape[0],-1)
x_test_vanilla = x_test.reshape(x_test.shape[0],-1)
print(x_train_vanilla.shape)

In [ ]:
#Vanilla Model
from keras.models import Sequential
from keras.layers import Dense,Dropout
model = Sequential()
model.add(Dense(1000,activation="relu",input_dim=x_train_vanilla.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(512,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(num_classes,activation="softmax"))
model.summary()

In [ ]:
#compile model
model.compile(loss="categorical_crossentropy",optimizer="rmsprop",
             metrics=["accuracy"])

In [ ]:
#Training the model!!
from keras.callbacks import ModelCheckpoint
#train model
checkpointer = ModelCheckpoint(filepath="MLP.weights.best.hdf5",verbose=1,
                              save_best_only=True)
hist = model.fit(x_train_vanilla,y_train,batch_size=32,epochs=20,
                validation_data=(x_valid_vanilla,y_valid),callbacks=[checkpointer],
                verbose=2,shuffle=True)

In [ ]:
#loading the weights that yeilds the best calidation accuracy
model.load_weights("MLP.weights.best.hdf5")

In [ ]:
##Checking the accuracy
accuracy=model.evaluate(x_test_vanilla,y_test)
print(100*accuracy[1],"Testing accuracy of the data in vanilla model")

# Second Model --CNNs

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
model = Sequential()
model.add(Conv2D(filters=16,activation="relu",padding="same",kernel_size=2,
                input_shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3])))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=32,activation="relu",padding="same",kernel_size=2))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,activation="relu",padding="same",kernel_size=2))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(510,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation="softmax"))
model.summary()

In [ ]:
#Compling the model
from keras import optimizers
adam = optimizers.Adam()
model.compile(loss="categorical_crossentropy",optimizer=adam,
             metrics=["accuracy"])

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpointer_cnn = ModelCheckpoint(filepath="CNN.best.weights.hdf5",
                                  verbose=1,save_best_only=True)
model.fit(x_train,y_train,verbose=2,batch_size=124,epochs=10,
          callbacks=[checkpointer_cnn],validation_data=(x_valid,y_valid),
         shuffle=True)

In [ ]:
model.load_weights("CNN.best.weights.hdf5")
score = model.evaluate(x_test,y_test)
print(100*score[1])

In [ ]:
print(100*score[1])

# Data Agumentation

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
checkpoint = ModelCheckpoint(filepath="CNN.aug_best.weights.hdf5",save_best_only=True,verbose=2)
datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,
                            rotation_range=20,width_shift_range=0.2,height_shift_range=0.2,
                            horizontal_flip=True)
datagen.fit(x_train)
hist = model.fit_generator(datagen.flow(x_train,y_train,batch_size=124),steps_per_epoch=len(x_train)/124,epochs=100,
                          callbacks=[checkpoint],validation_data = (x_valid,y_valid),verbose=2)

In [ ]:
model.load_weights("CNN.aug_best.weights.hdf5")
score = model.evaluate(x_test,y_test)
print(100*score[1])

In [ ]:
print(100*score[1])


## Create and Config Agumented Image Generator

In [ ]:
import skimage
from skimage import transform
x_train[1].shape
x_temp_128_18 = transform.resize(x_train[1],(128,18,3))
x_train = np.array([transform.resize(train,(128,18,3)) for train in x_train])
x_train.shape
x_valid = np.array([transform.resize(valid,(128,18,3)) for valid in x_valid])
x_test = np.array([transform.resize(test,(128,18,3)) for test in x_test])

In [ ]:
x_train.shape
x_valid.shape
plt.imshow(x_test[1])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen_train = ImageDataGenerator(width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  horizontal_flip = True)
datagen_train.fit(x_train)

In [ ]:
import matplotlib.pyplot as plt
#take a subset of training data
x_train_subset = x_train[:12]

#Visualize subset of taining data
fig = plt.figure(figsize=(20,2))
for i in range(0,len(x_train_subset)):
    ax = fig.add_subplot(1,12,i+1,xticks=[],yticks=[])
    ax.imshow(x_train_subset[i])
fig.suptitle("Subset of Original Training Images",fontsize=20)
plt.show()

# Visualize agumented images
fig = plt.figure(figsize=(20,2))
for x_batch in datagen_train.flow(x_train_subset,batch_size=12):
    for i in range(0,12):
        ax = fig.add_subplot(1,12,i+1,xticks=[],yticks=[])
        ax.imshow(x_batch[i])
    fig.suptitle("Agumented Images",fontsize=20)
    plt.show()
    break;

# Define the Model Architecture

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
model = Sequential()
model.add(Conv2D(filters=16,activation="relu",padding="same",kernel_size=(8,2),
                input_shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3])))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,activation="relu",padding="same",kernel_size=(8,2)))
model.add(MaxPooling2D(pool_size=(2)))
model.add(Conv2D(filters=64,activation="relu",padding="same",kernel_size=(8,2)))
model.add(MaxPooling2D(pool_size=(2)))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(num_classes,activation="softmax"))
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer="adam",
             metrics=["accuracy"])

In [ ]:
from keras.callbacks import ModelCheckpoint
batch_size=32
epochs =100
checkpointer=ModelCheckpoint(filepath="CNN.aug_best.weights.hdf5",
                            verbose=1,save_best_only=True)
model.fit_generator(datagen_train.flow(x_train,y_train,batch_size=batch_size),
                                      epochs=epochs,verbose=2,callbacks=[checkpointer],
                                      validation_data=(x_valid,y_valid),
                                      validation_steps=x_valid.shape[0]//batch_size,
                                      steps_per_epoch=x_train.shape[0]//batch_size)

# Visualizing CNNs
* So some CNNs works and others they don't!
* 

In [ ]:
from keras import applications
#build the VGG16 network
modle = applications.VGG16(include_top=False,
                          weights="imagenet")
#get the symbolic outputs of each "key" layer (we gave them unique name).
layer_dict = dict([(layer.name,layer) for layer in model.layers])

In [ ]:
from keras import backend as K
layer_name = "block5_conv3"
filter_index = 0 # can be any value form 0 to 511, as there are 512 filters in that layer 
#build a loss function that maximizes the activation
# of the nth filter of the layer considered
layer_output = layer_dict[layer_name].output
loss = K.mean(layer_output[:,:,:,filter_index])
#compute the gradient of the input picture wrt this loss
grads = K.gradients(loss,input_img)[0]
grads /= (K.sqrt(K.mean(K.square(grads)))+1e-5)
#this function returns the loss and grads given the input picture 
iterate = K.function([input_img],[loss,grads])

## AlexNet
* Pioneered the use of the ReLu activation function and dropout as a technique for avoiding overfitting
## VGG Architecture 
* Came from visual geometry group by Oxford University
* VGG 16
* VGG 19
* 3X3 convolutions
* Broken up by 2x2 pooling layers
* Finshed with 3 fully connected layers
* Pioneered the exclusive use of small 3x3 convolution windows(filter)
* AlexNet had 11x11 windows(which was way bigger than VGG Architecture)
## ResNet Archictecture 
* ResNet is kind of like VGG, but and not the same structure is repeated again and again for layer after layers like VGG
* The largest having a groundbreaking 152 layers.
* Previous researcher tried to have similar deeper layers but as performance increased for inital iterations but after some time it fell(performance declined) -- vanishing gradient problem.
* ResNet team added connections to their very deep CNN that skipped layers so that gradient signal has a shorter route to travel.

## Visualizing CNNs
* Each layer in the grid represents a pattern that causes the neuron in the first layer to activate-in other words, they are patterns that the first layer recognizes.
* Second layer in Cnn pick up more complex ideas like circles and stripes. 
* The Cnns learns to do this on its own.
* Third layer picks up complex combination of previous features that were picked up by previous layer.
* Last layer picks up most complex features.


## Transfer Learning
* Basic architecture of CNN - Input - CONV - CONV - POOL - CONV - CONV - POOL - CONV -CONV - POOL - DENSE -DENSE - DENSE 
* Why transfer learning work -- because the intial layer in Cnn detects features like edges and more deeper layer detect stuffs like circle square (shapes) and more deeper layer detect features which complex combination of intial features that is the reason **TRANSFER LEARNING** works!!
* Normal procedure for transfer learning is to remove the dense layers which are specific to the particular dataset and add new dense layer and train them(**only the dense layer**)!!
* For instance, the technique will work well if our data set is relatively small and very similar to ImageNet.
* Transfer Learning was used to develop a CNN to diagnose skin cancer
* The CNN classifies lesions as either benign or malignant and ** and achieves better performance than dermatologists for diagnosing some form of skin cancer**
* He used a transfer learning approach with the inception architecture pre-trained on ImageNet database.
* He removed the final densely connected classification layer and added a new fully connected layer.
* This layer had far fewer categories,one for each type of disease class that he wanted to detect.
* As of all the other layers in the network 
* Randomly initialize the weights in the new fully connected layer
* Intialize the rest of the weights using pre trained weights 
* Retrain the entire neural network
* SO model got a head start that it was given from pre-training on ImageNet.
* This approach with fine tuning the parameters in a network with different final classification layer,**works best if your data set is quite large and very different from ImageNet database**

## Transfer Learning -- notes
* Transfer learning involves taking pre trained neural network and adapting the neural network to a new, different data set.
* Depending to both:
* the size of the new data set, and 
* the similarity of the new data to the original data set
* the approach for usinf transfer learning will be different, There are four case:
    1. new data set is small, new data is similar to original training data set.
    2. new data set is small, new data is different from original training data
    3. new data set is large, new data is similar to original training data set.
    4. new data set is large, new data is different from original training dataset


* A large dataset -- one million image
* A small dataset -- two thousand images (matter of perspective)
* Overfitting is a concern when using transfer learning with a small dataset.
* images of dog and wolves are considered to be similar but image of dog and flower are not consider to be similar.
* Each of the four transfer learning cases has its own approach.
* So our example network contains three convolution layer and three dense fully connected layers.
* Generalized overview of what the convolution neural network does:
    * the first layer will detect edges in the image
    * the second layer will detect shapes
    * the third convolution layer detect higher level featutes 
* Each transfer learning case will use the pre trained convolution nn in a different way.


### Case 1 : **Small Data Set, Similar Data**
    * slice off the end the neural network
    * add a new fully connected layer that matches the number of classes in the new data set 
    * randomize the weights of the new fully connected layer; freeze all the weights from pre-trained network.
    * train the network to update the weights of the new fully connected layer.
* **to avoid overfitting on the small data set, the weights of the original network will be held constant then re-training the weights.**


### Case 2: ** Small Data Set, Different Data **
* If the new data set is small and different from the original training data:
    * slice off most of the pre-trained layers near the beginning of the network 
    * and to the remaining pre-trained layers a new fully connected layer that matches the number of classes in the new data set.
    * randomize the weights of the new fully conected layer; freeze all the weights from the pre-trained network.
    * train the network to update the weights of the new fully connected layer.
* Because the data set is small, overfitting is still a concern. To combat overfitting, the weights of the original neural network will be held constant, like in the first case,
* But the original training set and the new data set do not share higher level features.In this case, the network will only use the layers containing lower level features.


### Case 3: ** Large Data Set, Similar Data**
* if the new data set is large and similar to the orginal training data:
    * remove the last fully conncected layer and replace with a layer matching the number of classes in the new data set
    * randomly intialize the weights in the new fully connected layer 
    * initialize the rest of the weights using the pre-trained weights 
    * re -train the entire neural network
* Overfitting is not as much a concern when training a large data set; we can re-train all the weights
* Because the original training set and the new data set share higher level features, the entire neural network is used as well.


### Case 4: ** Large Data Set, Differernt Data**
* if the new data set is large and different from the orginial training data:
    * remove the last fully connected layer and replace with a layer matching the number of the classes in the new data set
    * retain the network from the scratch with randomly initilazied weights 
    * alternatively, you could just use the same strategy as the "large and similar" data case

* Even though the data set is different from taining data , initializing the weights from the pre-trained network might take training faster. So this case is exactly the same as the case with a large, similar data set.
* If using the pre-trained network as a starting point does not produce a successful model, another option is to randomly initialize the convolution neural network weights and train the network from scratch.

## Transfer Learning in Keras!!
* In CNN project -- the goal is to create an algorithm that can predict breed just from dog's picture!!
* We will be using the **VGG-16** model pre-trained on the ImageNet data set.
* The dog data set is relatively small and it has signficant overlap with a subset of ImageNet categories.
* SO this condition is -- similar data & small Data Set so we have to keep in mind not overfit while doing the transfer learning so we will remove last fully connected dense layer and randomize its weight and train it and freeze other weights so it dosent overfit!!
* So we will use keras..
    ## (224,224,3) --> VGG-16 --> (7,7,512) 
    ##          (Conv and pooling layer)
* then we can save the output of this as a new dataset 
* and we will make our own fully connected dense layer according to our output calssification problem!!
* need to remember this model.add(Flatten(input_shape=(7,7,512)))

# 1. Load and Preprocess Sample Images

We will be loading the pre-trained weights from ImageNet of VGG16 architecture and the pre-process step are as following.
* We have imported a very small dataset of 8 images and stored the preprocessed image input as img_input. Note that the dimensionality of this array is (8,224,224,3).In this case, each of the 8 images is a 3D tensor, with shape (224,224,3).

In [ ]:
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image 
import numpy as np 
import glob
img_paths =glob.glob("images/*.jpg")
def path_to_tenser(img_path):
    #load RBG image as PIL.Image type 
    img = image.load_img(img_path,target_size = (224,224))
    # convert PIL.image.image type to 3D tensor with shape (224,224,3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1,224,224,3)
    return np.expand_dims(x,axis=0)
def paths_to_tensor(img_paths):
    list_of_tensor = [path_to_tensor(img_path) for img_path in img_paths]
    return np.vstack(list_of_tensor)
# calculate the image input. you will learn more about how this works in project!
#img_input = preprocess_input(paths_to_tensor(img_paths))
#print(img_input.shape)

In [ ]:
#from keras.applications.vgg16 import VGG16
#model = VGG16()
#model.summary()

# Tensorflow -- Convolution Layers

In [ ]:
import tensorflow as tf
# output depth 
k_output = 64
#image dimensions
image_width = 10
image_height = 10
color_channels = 3
# convolution filter dimension
filter_size_width = 5
filter_size_height = 5
#input/image
input = tf.placeholder(tf.float32,shape=[None,image_height,image_width,color_channels])
# weights and bias 
weight = tf.Variable(tf.truncated_normal([filter_size_height,
                                         filter_size_width,color_channels,
                                         k_output]))
bias = tf.Variable(tf.zeros(k_output))

#apply convolution
conv_layer = tf.nn.conv2d(input,weight,strides=[1,2,2,1],padding="SAME")
## add bias 
conv_layer = tf.nn.bias_add(conv_layer,bias)
# apply activation function 
conv_layer = tf.nn.relu(conv_layer)

The code above use the **tf.nn.conv2d()** function ato compute the convolution with weight as the **filters** and [1,2,2,1] for the strides
* TensorFlow **uses a stride for each input dimension,** [batch,input_height,input_width,input_channels]
* we generally always set the strides for batch and input_channels(i.e. the first and fourth element in the strides array) to be 1. this ensure that the model uses all the batch and input_channels. *(Its good practice to remove the batches or channnels you want to skip from the data set rather use a stride to skip them)*
* we will focus on changing input_height and input_width strides are for striding the filter over input.
    * IF using strides of two then 
    * **tf.nn.conv(x,W,strides=[1,2,2,1]**
    * better to use a tf.nn.bias_add()
    
    
* **Things to remember are:**
    * the shape of weight(filter) is [filter_height,filter_width,depth_of_input,number_feeatures_you_want_to_detect]
    * [5,5,3,64] so this a filter for first conv layer whose input is **rbg** data set and we want depth of conv layer to be 64(64 pattern to detect)


## Using Convolution Layers in TensorFlow
Let's now build a convoultion layer in TensorFlow. In the below exercis , you'll be asked to set up dimensions of convolution filters, weights, and the biases. This is in many ways the trickiest part using CNNsin TensorFlow. Once you have a sense of how to set up the dimensions of these attributes, applying CNNs will be far more straightforward.

## Review 
You should go over the TensorFlow documentation for 2D convolutions. Most of the documentation is straightforward, except perhaps the padding argument. The padding might differ depending on wheteher you pass "VALID" and "SAME".

Here are a few more things worth reviewing:
   1. Introduction of TensorFlow
   2. How to **determine the dimensions of the output based on the input size and the filter size!**
       * new_height = (input_height - filter_height + 2 x P)/S +1
       * new_width = (input_width - filter +2 x P )/S +1
       * "VALID" **--NO PADDING**
       * "SAME" **--PADDING**
       * out_height = **ceil(float(in_height-filter_height+1)/float(strides[1]))**
       * out_width = **ceil(float(in_width - filter_Width+1)/float(strides[2]))**
       
        

## Max Pooling Layers in TensorFlow
* Function -- **tf.nn.max_pool()**


In [ ]:
conv_layer = tf.nn.conv2d(input,weight,strides=[1,2,2,1],padding="SAME")
conv_layer = tf.nn.bias_add(conv_layer,bias)
conv_layer = tf.nn.relu(conv_layer)
# apply max pooling layer
conv_layer = tf.nn.max_pool(conv_layer,ksize=[1,2,2,1],
                           strides = [1,2,2,1],padding="SAME")

* Function -- **tf.nn.max_pool()**
* Arguments
    * ksize
    * strides
    * format -- [batch,height,width,channels]
    * for both ksize and strides the batch and channel dimension are typically set to 1.

## Convolution Neural Networks in TensorFlow
* Basic architecture of "ALEXNET"
* Image -> Convolution -> max_pooling -> Convolution -> max_pooling -> fully_connected -> fully_connected -> calssifier

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".",one_hot =True,reshape=False)

import tensorflow as tf

# parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128 

# number of samples to calculate validation and accuracy
# decrease this if you're running out of memory
test_valdi_size = 256

# network Parameters
n_classes = 10 # MNIST total calsses (0-9 digits)
dropout =0.75 # dropout (probability to keep units)


In [ ]:
# store weights and biases

weight = {
    "wc1":tf.Variable(tf.random_normal([5,5,1,32])),#because its black and weight so the 3rd position is 1"
    "wc2":tf.Variable(tf.random_normal([5,5,32,64])),
    "wd1":tf.Variable(tf.random_normal([7*7*64,1024])),
    "out": tf.Variable(tf.random_normal([1024,n_classes]))
}
biases = {
    "bc1":tf.Variable(tf.random_normal([32])),
    "bc2":tf.Variable(tf.random_normal([64])),
    "bd1":tf.Variable(tf.random_normal([1024])),
    "out":tf.Variable(tf.random_normal([n_classes]))
}

## Convolution Layer
* **tf.nn.conv2d()**
* **tf.nn.bias_add()**
* **tf.nn.relu()**

In [ ]:
def conv2d(x,W,b,strides=1):
    x = tf.nn.conv2d(x,W,strides = [1,strides,strides,1],padding ="same")
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

## Max Pooling Layers
* **tf.nn.max_pool()**

In [ ]:
def maxpool2d(x,k=2):
    return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],
                         padding = "same")

## Model
* The transformation of each layer to new dimensions is shown in the comments, the first layer shapes the images from 28x28x1 to 28x28x32 in the convolution step . The next step applies max pooling, turning each sample into 14x14x32.All layer are applied from conv1 to output, producing 10 class prediction

In [ ]:
def conv_net(x,weights,biases,dropout):
    # Layer 1 - 28x28x1 to 14x14x32
    conv1 = conv2d(x,weights["wc1"],biases["bc1"])
    conv1 = maxpool2d(conv2,k=2)
    
    # Layer 2 - 14x14x32 to 7x7x64
    conv2 = conv2d(conv1,weights["wc2"],baises["bc2"])
    conv2 = maxpool2d(conv2,k=2)
    
    # Fully Connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2,[-1,weights["wd1"].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1,weights["wd1"]),biases["bd1"])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1,dropout)
    
    # Output Layer - class prediction - 1024 to 10
    out - tf.add(tf.matmul(fc1,weights["out"]),biases["out"])
    return out

## Session

In [ ]:
# tf Graph input 
x = tf.placeholder(tf.float32,[None,28,28,1])
y = tf.placeholder(tf.float32,[None,n_classes])
keep_prob = tf.placeholder(tf.float32)

#Model 
logits = conv_net(x,weights,biases,keep_prob)

# Define Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                             labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

# Initializing the variables 
init = tf.global_variables_initializer()

## Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer,feed_dict = {x:batch_x,
                                           y:batch_y,
                                           keep_prob:dropout})
            # Calculate batch loss and accuracy
            loss = sess.run(cost,feed_dict = {x:batch_x,
                                             y:batch_y,
                                             keep_porb:1.0})
            valid_acc = sess.run(accuracy, feed_dict = { 
            x:mnist.validation.images[:test_valid_size],
            y:mnist.validation.labels[:test_valid_size],
            keep_prob = 1.0})
            
            print("Epoch {:>2}, Batch {:>3} -"
                 "loss: {:10.4f} Validation Accuracy: {:.6f}".format(
                 epoch+1,
                 batch+1,
                 loss,
                 valid_acc))
            
            #Calculate Test Accuracy 
            test_acc = sess.run(accuracy,feed_dict={
                x:mnist.test.images[:test_valid_size],
                y:mnist.test.labels[:test_valid_size],
                keep_prob =1.0
            })
            print("Testing Accuracy: {}".format(test_acc))



# Weight Initializing Intro

The general rule for setting the weights in a neural network is to be  close to zero without being too small. A good practice is to start your weight in the range [-y,y] where y = 1/sqrt(n) (n is the number of input to a given neuron).
* **tf.random_normal()**
* **tf.random_uniform()**
* so truncated normal distribution is when we put thershold limit to our distribution beyond that limit no random number should be generated.
* **tf.truncated_normal(shape,mean=0,std=1.0)**
* So this function generate normal distributed values and drop any |values| which are greater than 2.(by default)


# AutoEncoder

These models are used to compress data, as well as image denoising, which we'll be implementing in this lesson. The idea here is we'll build a network that tries to generate it's input data,but with a narrow hidden layer that serves as a compressed representation of the input data.
* This is a type of network architecture to perform **data compression**, where the compression and decompression **functions are learned from the data itself**, not hand-engineered by humans.
* So **encoder** and **decoder** are build with data itself.
* The whole network is trained by **minimizing** the difference between the **input and output**.
* STRUCTURE OF AUTOENCODER
* **
* **[10] -> [5] -> [2][BOTTEL NECK HIDDEN LAYER) -> (5) -> (10)**
* **
* Bottel neck hidden layer is the compressed data form which we can create our original data.
* There will be some loss of information!!(But very less)
* So autoencoder are **not good(WORST)** in :
    1. **COMPRESSION**(compared to JEPGs,MP3s and MPEgs)
    2. **GENERALIZING TO DATASETS**(other than what they were trained on)
* Application of AutoEncoder:
    1. **IMAGE DENOISING**
    2. **DIMENSIONALITY REDUCTION**
* so now we will compress a image(using autoencoder) then sice its a image data we'll improve it by using convolutional layers    
* We will learn how to code autoencoder on tensorflow


## AutoEncoder with tensorflow

In [ ]:
##Importing data 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data",validation_size=0)

In [ ]:
img = mnist.train.images[2]
plt.imshow(img.reshape((28,28)),cmap ="gray")
plt.axis("off")
plt.show()

In [ ]:
mnist.train.images[0].shape


In [ ]:
n_hidden = 392
n_hidden_2 = 32
weight = {
    "w1":tf.Variable(tf.truncated_normal([mnist.train.images.shape[1],n_hidden])),
    "w2":tf.Variable(tf.truncated_normal([n_hidden,n_hidden_2])),
    "w3":tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden])),
    "w4":tf.Variable(tf.truncated_normal([n_hidden,mnist.train.images.shape[1]]))
}
biases = {
    "b1":tf.Variable(tf.zeros([n_hidden])),
    "b2":tf.Variable(tf.zeros([n_hidden_2])),
    "b3":tf.Variable(tf.zeros([n_hidden])),
    "b4":tf.Variable(tf.zeros([mnist.train.images.shape[1]]))
}

In [ ]:
def forward_path(x,weight,biases):
    pass_1 = tf.add(tf.matmul(x,weight["w1"]),biases["b1"])
    pass_1 = tf.nn.relu(pass_1)
    pass_2 = tf.add(tf.matmul(pass_1,weight["w2"]),biases["b2"])
    pass_2 = tf.nn.relu(pass_2)
    encoded = pass_2
    pass_3 = tf.add(tf.matmul(pass_2,weight["w3"]),biases["b3"])
    pass_3 = tf.nn.relu(pass_3)
    pass_4 = tf.add(tf.matmul(pass_1,weight["w4"]),biases["b4"])
    return pass_4,encoded

In [ ]:
sess = tf.Session()

In [ ]:
# Creating placeholders
batch_size = 128
epochs = 30
n_classes = mnist.train.images.shape[1]
x =tf.placeholder(tf.float32,[None,n_classes])
y = tf.placeholder(tf.float32,[None,n_classes])

logits,encoded = forward_path(x,weight,biases)

cost = tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=logits)
cost = tf.reduce_mean(cost)
decoded = tf.nn.sigmoid(logits)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

init = tf.global_variables_initializer()


sess.run(init)
for epoch in range(epochs):
    for batch in range(mnist.train.num_examples//batch_size):
        batch_x = mnist.train.next_batch(batch_size)
        _,loss=sess.run([optimizer,cost],feed_dict = {x:batch_x[0],y:batch_x[0]})
            #loss = sess.run(cost,feed_dict = {x:batch_x[0]})
            
            
    print("Epoch {:>2}, Batch {:>3} -"
                "loss: {:10.4f}".format(
                 epoch+1,
                 batch+1,
                 loss))
    
    



In [ ]:

fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(20,4))
in_imgs = mnist.test.images[:10]
reconstructed, compressed = sess.run([decoded, encoded], feed_dict={x: in_imgs})

for images, row in zip([in_imgs, reconstructed], axes):
    for img, ax in zip(images, row):
        ax.imshow(img.reshape((28, 28)), cmap='Greys_r')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

fig.tight_layout(pad=0.1)
plt.show()

In [ ]:
sess.close()